In [90]:
import pandas as pd
import numpy as np
from database import engine

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [102]:
def query_data():
    query = """
    select
	    strdrink,
	    stringredient1,
	    stringredient2,
	    stringredient3,
	    stringredient4,
	    stringredient5,
	    stringredient6,
	    stringredient7,
	    stringredient8,
	    stringredient9,
	    stringredient10,
	    stringredient11,
	    stringredient12,
	    strmeasure1_clean,
	    strmeasure2_clean,
	    strmeasure3_clean,
	    strmeasure4_clean,
	    strmeasure5_clean,
	    strmeasure6_clean,
	    strmeasure7_clean,
	    strmeasure8_clean,
	    strmeasure9_clean,
	    strmeasure10_clean,
	    strmeasure11_clean,
        strmeasure12_clean
	    
	from 
	    all_cocktails
	;
        """

    return pd.read_sql_query(query, engine)

<IPython.core.display.Javascript object>

In [103]:
df = query_data()

# display(df.isna().sum())
# display(df.stralcoholic.value_counts())
# df.strglass.value_counts()
# df.info()

<IPython.core.display.Javascript object>

In [104]:
ingred_cols = [col for col in df.columns if col.startswith("stringredient")]
measure_cols = [col for col in df.columns if col.startswith("strmeasure")]

<IPython.core.display.Javascript object>

In [109]:
# Convert all ingredients to lowercase
df[ingred_cols] = df[ingred_cols].apply(lambda col: col.str.lower())


# Get data in a usable format for analysis
ingred_long = (
    df[["strdrink"] + ingred_cols]
    .melt(id_vars="strdrink", value_vars=ingred_cols)
    .assign(
        ingred_num=lambda df_: df_["variable"]
        .str.replace("stringredient", "")
        .astype("int")
    )
    .drop("variable", axis=1)
    .rename({"value": "ingredient"}, axis=1)
)

measure_long = (
    df[["strdrink"] + measure_cols]
    .melt(id_vars="strdrink", value_vars=measure_cols)
    .assign(
        ingred_num=lambda df_: df_["variable"]
        .str.replace("strmeasure", "")
        .str.replace("_clean", "")
        .astype("int")
    )
    .drop("variable", axis=1)
    .rename({"value": "amount"}, axis=1)
)

combo = (
    ingred_long.merge(measure_long, on=["strdrink", "ingred_num"])
    .sort_values(["strdrink", "ingred_num"])
    .drop("ingred_num", axis=1)
    .dropna()
    .reset_index(drop=True)
    .pivot_table(
        index="strdrink", columns="ingredient", values="amount", aggfunc=np.sum
    )
    .fillna(0)
    .reset_index()
)

combo

ingredient,strdrink,151 proof rum,7-up,absinthe,absolut citron,absolut kurant,absolut peppar,absolut vodka,advocaat,agave syrup,...,white rum,white wine,wild turkey,wine,worcestershire sauce,wormwood,yellow chartreuse,yoghurt,yukon jack,zima
0,1-900-FUK-MEUP,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,110 in the shade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,151 Florida Bushwacker,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,155 Belmont,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24k nightmare,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,Zizi Coin-coin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
624,Zoksel,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
625,Zombie,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
626,Zorbatini,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

Left off here - the final two steps to make this work (before checking it that it is actually right) are as follows:
(2) Need to calculate row sums and get proportions for each drink

After that, we have the data matrix and need to figure out a good way to visualize or analyze this (Ed mentioned doing some Cos analysis)